In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import DataFrame
spark = SparkSession.builder \
    .appName("fill_account_balance_f_31.12.2017") \
    .enableHiveSupport() \
    .getOrCreate()

In [2]:
ft_balance_f = spark.read.parquet('/user/hive/warehouse/ds.db/ft_balance_f')
ft_balance_f.show()

+----------+----------+-----------+--------------+
|   ON_DATE|ACCOUNT_RK|CURRENCY_RK|   BALANCE_OUT|
+----------+----------+-----------+--------------+
|31.12.2017|  36237725|         35|      38318.13|
|31.12.2017|     24656|         35|      80533.62|
|31.12.2017|  18849846|         34|      63891.96|
|31.12.2017|   1972647|         34|     5087732.1|
|31.12.2017|  34157174|         34|     7097806.9|
|31.12.2017|  48829156|         34|      87620.47|
|31.12.2017|     13905|         34|      129554.0|
|31.12.2017|     17244|         34|    2025852.49|
|31.12.2017|  34156787|         31|      32640.33|
|31.12.2017|     17132|         34| 2.001137158E7|
|31.12.2017|     13560|         34|    1055629.43|
|31.12.2017|     17439|         34|     130813.42|
|31.12.2017|     17434|         34|    5008133.82|
|31.12.2017|     13630|         34|2.0007965012E8|
|31.12.2017| 277937699|         34|      35341.17|
|31.12.2017|     13906|         30|     194729.89|
|31.12.2017|     14136|        

In [4]:
md_exchange_rate_d = spark.read.parquet('/user/hive/warehouse/ds.db/md_exchange_rate_d')
md_exchange_rate_d.show()

+----------------+--------------------+-----------+--------------+------------+
|DATA_ACTUAL_DATE|DATA_ACTUAL_END_DATE|CURRENCY_RK|REDUCED_COURCE|CODE_ISO_NUM|
+----------------+--------------------+-----------+--------------+------------+
|      2016-07-01|          2050-12-31|         38|       31.8884|         974|
|      2018-01-31|          2018-01-31|  427870281|       28.3798|         933|
|      2018-01-31|          2018-01-31|         29|       7.11613|         752|
|      2018-01-31|          2018-01-31|  529511970|       33.0543|         944|
|      2018-01-31|          2018-01-31|         62|       8.87555|         156|
|      2018-01-31|          2018-01-31|  205699733|       2.74512|         203|
|      2018-01-31|          2018-01-31|  205699734|      0.224063|         348|
|      2018-01-31|          2018-01-31|  409611215|       6.37718|         972|
|      2018-01-31|          2018-01-31|         55|     0.0524955|         410|
|      2018-01-31|          2018-01-31| 

In [8]:
ruble_exchange_row = md_exchange_rate_d \
    .filter(F.col("CODE_ISO_NUM") == 643) \
    .select("REDUCED_COURCE") \
    .limit(1) \
    .collect()

if ruble_exchange_row:
    ruble_exchange_rate = ruble_exchange_row[0]["REDUCED_COURCE"]
else:
    ruble_exchange_rate = 1.0

+--------------+
|REDUCED_COURCE|
+--------------+
+--------------+



In [11]:
dm_account_balance_f = ft_balance_f.where(F.col('on_date') == '31.12.2017')\
.select('on_date', 'account_rk', 'balance_out', (F.col('balance_out') * ruble_exchange_rate).alias('balance_out_rub'))

In [12]:
dm_account_balance_f.show()

+----------+----------+--------------+---------------+
|   on_date|account_rk|   balance_out|balance_out_rub|
+----------+----------+--------------+---------------+
|31.12.2017|  36237725|      38318.13|       38318.13|
|31.12.2017|     24656|      80533.62|       80533.62|
|31.12.2017|  18849846|      63891.96|       63891.96|
|31.12.2017|   1972647|     5087732.1|      5087732.1|
|31.12.2017|  34157174|     7097806.9|      7097806.9|
|31.12.2017|  48829156|      87620.47|       87620.47|
|31.12.2017|     13905|      129554.0|       129554.0|
|31.12.2017|     17244|    2025852.49|     2025852.49|
|31.12.2017|  34156787|      32640.33|       32640.33|
|31.12.2017|     17132| 2.001137158E7|  2.001137158E7|
|31.12.2017|     13560|    1055629.43|     1055629.43|
|31.12.2017|     17439|     130813.42|      130813.42|
|31.12.2017|     17434|    5008133.82|     5008133.82|
|31.12.2017|     13630|2.0007965012E8| 2.0007965012E8|
|31.12.2017| 277937699|      35341.17|       35341.17|
|31.12.201